<a href="https://colab.research.google.com/github/scarabinoalbano/03MIAR_04_A_2025-26_Algoritmos-de-Optimizacion/blob/main/AlbanoScarabino_ProyectoDeProgramacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algoritmos de optimización - Seminario**<br>
Nombre y Apellidos: Albano Scarabino <br>
Colab: https://colab.research.google.com/drive/1YWLnClzIomMdfKMZatzOaDCGCygpKgMZ?usp=sharing <br>
Url: https://github.com/scarabinoalbano/03MIAR_04_A_2025-26_Algoritmos-de-Optimizacion/blob/main/AlbanoScarabino_ProyectoDeProgramacion.ipynb

---
# **Problema 1: Organizar sesiones de doblaje**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:
- Número de actores: 10
- Número de tomas: 30
- Actores/Tomas: https://bit.ly/36D8IuK
   - 1 indica que el actor participa en la toma
   - 0 en caso contrario

In [1]:
matriz_enunciado = [
        [1,1,1,1,1,0,0,0,0,0],  # Toma 1
        [0,0,1,1,1,0,0,0,0,0],  # Toma 2
        [0,1,0,0,1,0,1,0,0,0],  # Toma 3
        [1,1,0,0,0,0,1,1,0,0],  # Toma 4
        [0,1,0,1,0,0,0,1,0,0],  # Toma 5
        [1,1,0,1,1,0,0,0,0,0],  # Toma 6
        [1,1,0,1,1,0,0,0,0,0],  # Toma 7
        [1,1,0,0,0,1,0,0,0,0],  # Toma 8
        [1,1,0,1,0,0,0,0,0,0],  # Toma 9
        [1,1,0,0,0,1,0,0,1,0],  # Toma 10
        [1,1,1,0,1,0,0,1,0,0],  # Toma 11
        [1,1,1,1,0,1,0,0,0,0],  # Toma 12
        [1,0,0,1,1,0,0,0,0,0],  # Toma 13
        [1,0,1,0,0,1,0,0,0,0],  # Toma 14
        [1,1,0,0,0,0,1,0,0,0],  # Toma 15
        [0,0,0,1,0,0,0,0,0,1],  # Toma 16
        [1,0,1,0,0,0,0,0,0,0],  # Toma 17
        [0,0,1,0,0,1,0,0,0,0],  # Toma 18
        [1,0,1,0,0,0,0,0,0,0],  # Toma 19
        [1,0,1,1,1,0,0,0,0,0],  # Toma 20
        [0,0,0,0,0,1,0,1,0,0],  # Toma 21
        [1,1,1,1,0,0,0,0,0,0],  # Toma 22
        [1,0,1,0,0,0,0,0,0,0],  # Toma 23
        [0,0,1,0,0,1,0,0,0,0],  # Toma 24
        [1,1,0,1,0,0,0,0,0,1],  # Toma 25
        [1,0,1,0,1,0,0,0,1,0],  # Toma 26
        [0,0,0,1,1,0,0,0,0,0],  # Toma 27
        [1,0,0,1,0,0,0,0,0,0],  # Toma 28
        [1,0,0,0,1,1,0,0,0,0],  # Toma 29
        [1,0,0,1,0,0,0,0,0,0]   # Toma 30
]
max_tomas_por_dia_enunciado = 6

---
# **¿Cúantas posibilidades hay sin tener en cuenta las restricciones?**

## Fórmula con restricción de tomas por día (k)

Sean:

- `( n )`: número total de tomas  
- `( k )`: máximo de tomas por día  
- `( d )`: número de días  
- `( R(n, d, k) )`: número de formas de particionar `( n )` tomas en `( d )` días, con hasta `( k )` tomas por día (considerando orden)

#### Fórmula General (sin orden de los días)

$$
P(n, k) = \sum_{d = \lceil n/k \rceil}^{n} \frac{R(n, d, k)}{d!}
$$

#### Recurrencia para \( R(n, d, k) \)

$$
R(n, d, k) =
\begin{cases}
1 & \text{si } n = 0 \text{ y } d = 0 \\\\
0 & \text{si } n < 0 \text{ o } d = 0 \text{ o } n > d \cdot k \\\\
\sum_{i = 1}^{\min(k, n - d + 1)} \binom{n}{i} \cdot R(n - i, d - 1, k) & \text{en otro caso}
\end{cases}
$$

## Fórmula sin restricción de tomas por día (k → ∞)

Cuando no hay restricción de cuántas tomas pueden hacerse por día, el número total de particiones posibles del conjunto de `n` tomas es dado por el **n-ésimo número de Bell** \( B_n \):

#### Fórmula general:

$$
P(n, \infty) = B_n
$$

donde \( B_n \) es el número de Bell, que cuenta las formas de particionar un conjunto de `n` elementos en subconjuntos no vacíos (sin importar el orden de los subconjuntos).

#### Recurrencia:

$$
B_0 = 1 \\
B_{n+1} = \sum_{k=0}^{n} \binom{n}{k} B_k
$$

In [2]:
import math

def bell_number(n):
    B = [0] * (n + 1)
    B[0] = 1
    for i in range(1, n + 1):
        B[i] = sum(math.comb(i - 1, k) * B[k] for k in range(i))
    return B[n]

# Ejemplo para n = 30
print(bell_number(30))

846749014511809332450147


---
# **¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?**

## Ejemplo de cálculo para n = 30, k = 6

Queremos calcular:

$$
P(30, 6) = \sum_{d = \lceil 30/6 \rceil}^{30} \frac{R(30, d, 6)}{d!}
$$

Es decir:

- Mínimo número de días: \( \lceil 30/6 \rceil = 5 \)
- Máximo número de días: 30
- Entonces se calcula:

$$
P(30, 6) = \frac{R(30, 5, 6)}{5!} + \frac{R(30, 6, 6)}{6!} + \cdots + \frac{R(30, 30, 6)}{30!}
$$


In [3]:
import math
from functools import lru_cache

@lru_cache(maxsize=None)
def R(n, d, k):
    """
    Calcula la cantidad de particiones ordenadas de `n` tomas en `d` días,
    donde cada día tiene entre 1 y `k` tomas como máximo.

    Parámetros:
    - n (int): número total de tomas
    - d (int): número exacto de días
    - k (int): número máximo de tomas por día

    Retorna:
    - int: número de particiones posibles (ordenadas)
    """
    if n == 0 and d == 0:
        return 1
    if n < 0 or d == 0 or n > d * k:
        return 0

    total = 0
    for i in range(1, min(k, n - d + 1) + 1):
        total += math.comb(n, i) * R(n - i, d - 1, k)
    return total

def total_posibilidades_sin_orden(n, k):
    """
    Calcula el número total de particiones válidas de `n` tomas en días,
    donde cada día tiene entre 1 y `k` tomas, sin importar el orden de los días.

    Parámetros:
    - n (int): número total de tomas
    - k (int): número máximo de tomas por día

    Retorna:
    - int: total de particiones distintas (sin contar repeticiones por orden)
    """
    d_min = math.ceil(n / k)
    total = 0
    for d in range(d_min, n + 1):
        total += R(n, d, k) // math.factorial(d)
    return total

# Definir rangos para n y k
rangos_n = list(range(4, 31))  # Del 4 al 30 inclusive
rangos_k = [3, 4, 5, 6, 7, 8]

print("Análisis de iteraciones para diferentes valores de n y k")
print("=" * 60)
print(f"{'n':>3} | {'k':>3} | {'Iteraciones':>15}")
print("-" * 30)

# Doble bucle para barrer todos los valores
for n in rangos_n:
    for k in rangos_k:
            total_iteraciones = total_posibilidades_sin_orden(n, k)
            print(f"{n:>3} | {k:>3} | {total_iteraciones:>15,}")
    print("-" * 30)  # Separador entre diferentes valores de n

Análisis de iteraciones para diferentes valores de n y k
  n |   k |     Iteraciones
------------------------------
  4 |   3 |              14
  4 |   4 |              15
  4 |   5 |              15
  4 |   6 |              15
  4 |   7 |              15
  4 |   8 |              15
------------------------------
  5 |   3 |              46
  5 |   4 |              51
  5 |   5 |              52
  5 |   6 |              52
  5 |   7 |              52
  5 |   8 |              52
------------------------------
  6 |   3 |             166
  6 |   4 |             196
  6 |   5 |             202
  6 |   6 |             203
  6 |   7 |             203
  6 |   8 |             203
------------------------------
  7 |   3 |             652
  7 |   4 |             827
  7 |   5 |             869
  7 |   6 |             876
  7 |   7 |             877
  7 |   8 |             877
------------------------------
  8 |   3 |           2,780
  8 |   4 |           3,795
  8 |   5 |           4,075
  8 

---
# **¿Cuál es la función objetivo?**

Minimizar la cantidad total de actores involucrados a lo largo de todos los días:

$$
\text{Minimizar: } f(x) = \sum_{d=1}^{D} |A_d|
$$

Donde:

- $D$: número total de días  
- $A_d$: conjunto de actores únicos necesarios en el día $d$  
- $|A_d|$: número de actores únicos en el día $d$ (cardinalidad del conjunto $A_d$)

In [4]:
def funcion_objetivo(planificacion, matriz_actores_tomas):
    """
    Función objetivo: minimizar el costo total de actores
    """
    costo_total = 0
    for dia in planificacion:
        actores_del_dia = set()
        for toma in dia:
            for actor in range(len(matriz_actores_tomas[0])):
                if matriz_actores_tomas[toma][actor] == 1:
                    actores_del_dia.add(actor)
        actores_dia = len(actores_del_dia)
        costo_total += actores_dia  # Suma el número de actores únicos por día
    return costo_total

---
# **¿Es un problema de maximización o de minimización?**

Es definitivamente un problema de minimización donde se busca la planificación con el menor costo total de actores-día.

Sujeto a:
- Cada toma se asigna a exactamente un día
- Máximo 6 tomas por día
- Todos los actores necesarios están disponibles cada día

---
# **Diseña un algoritmo para resolver el problema por fuerza bruta.**

In [5]:
import time

def generar_todas_particiones(tomas, max_tomas_por_dia):
    """
    Genera todas las posibles particiones de tomas en días usando fuerza bruta
    """
    def particionar_recursivo(tomas_restantes, particion_actual):
        if not tomas_restantes:
            yield particion_actual[:]
            return

        # Probar agregar la siguiente toma a cada día existente (si cabe)
        for i, dia in enumerate(particion_actual):
            if len(dia) < max_tomas_por_dia:
                dia.append(tomas_restantes[0])
                yield from particionar_recursivo(tomas_restantes[1:], particion_actual)
                dia.pop()

        # Probar crear un nuevo día con la siguiente toma
        particion_actual.append([tomas_restantes[0]])
        yield from particionar_recursivo(tomas_restantes[1:], particion_actual)
        particion_actual.pop()

    yield from particionar_recursivo(tomas, [])

def funcion_objetivo(planificacion, matriz_actores_tomas):
    """
    Función objetivo: minimizar el costo total de actores
    """
    costo_total = 0
    for dia in planificacion:
        actores_del_dia = set()
        for toma in dia:
            for actor in range(len(matriz_actores_tomas[0])):
                if matriz_actores_tomas[toma][actor] == 1:
                    actores_del_dia.add(actor)
        actores_dia = len(actores_del_dia)
        costo_total += actores_dia  # Suma el número de actores únicos por día
    return costo_total

def fuerza_bruta_doblaje(matriz_actores_tomas, max_tomas_por_dia):
    """
    Resuelve el problema de doblaje usando fuerza bruta pura
    """
    tomas = list(range(len(matriz_actores_tomas)))
    mejor_costo = float('inf')
    mejor_planificacion = None
    iteraciones = 0

    print("\nGenerando todas las particiones posibles...")

    for particion in generar_todas_particiones(tomas, max_tomas_por_dia):
        iteraciones += 1

        # Calcular costo total usando la función objetivo
        costo_total = funcion_objetivo(particion, matriz_actores_tomas)

        # Actualizar mejor solución si es necesario
        if costo_total < mejor_costo:
            mejor_costo = costo_total
            mejor_planificacion = [dia[:] for dia in particion]
            print(f"Nueva mejor solucion encontrada - Costo: {mejor_costo}, Dias: {len(particion)}")

        # Mostrar progreso cada cierto número de iteraciones
        if iteraciones % 100000 == 0:
            print(f"Iteraciones: {iteraciones:}, Mejor costo: {mejor_costo}")

    return mejor_planificacion

def mostrar_planificacion(planificacion, matriz_actores_tomas):
    """
    Muestra la planificación de manera detallada
    """
    if not planificacion:
        print("No se encontro planificacion")
        return

    print("\n=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===")

    costo_total = 0
    num_actores = len(matriz_actores_tomas[0])

    for i, dia in enumerate(planificacion, 1):
        print(f"\nDIA {i}:")
        print(f"Tomas: {[t+1 for t in dia]}")  # +1 para mostrar tomas desde 1

        # Calcular actores únicos del día
        actores_del_dia = set()
        for toma in dia:
            for actor in range(num_actores):
                if matriz_actores_tomas[toma][actor] == 1:
                    actores_del_dia.add(actor + 1)  # +1 para mostrar actores desde 1

        actores_ordenados = sorted(list(actores_del_dia))
        print(f"Actores necesarios: {actores_ordenados}")
        print(f"Costo del dia: {len(actores_ordenados)}")

        costo_total += len(actores_ordenados)

    print(f"\nCOSTO TOTAL: {costo_total}")
    print(f"DIAS TOTALES: {len(planificacion)}\n")

def ejecutar_algoritmo_fuerza_bruta(matriz_actores_tomas, max_tomas_por_dia):
    """
    Función principal para resolver el problema de doblaje con una matriz personalizada
    """
    print("="*60)
    print("ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA")
    print("="*60)

    total_iteraciones = total_posibilidades_sin_orden(len(matriz_actores_tomas), max_tomas_por_dia)
    print(f"\nSe realizarán {total_iteraciones} iteraciones.")

    # Ejecutar algoritmo de fuerza bruta
    tiempo_inicio = time.time()
    mejor_planificacion = fuerza_bruta_doblaje(matriz_actores_tomas, max_tomas_por_dia)
    tiempo_total = time.time() - tiempo_inicio
    print(f"Tiempo: {tiempo_total:.2f} segundos")

    # Mostrar resultados
    mostrar_planificacion(mejor_planificacion, matriz_actores_tomas)

In [6]:
# Es computacionalmente imposible resolver n=30, k=6 con fuerza bruta.
total_iteraciones = total_posibilidades_sin_orden(len(matriz_enunciado), max_tomas_por_dia_enunciado)
print(f"\nSe necesitan {total_iteraciones} iteraciones.")


Se necesitan 726391948970868949621309 iteraciones.


---
# **Se aplica el algoritmo por fuerza bruta a algunos ejemplos**

In [7]:
matriz = [
    [1, 1, 0, 0],  # Toma 1: Actores 1 y 2
    [0, 1, 1, 0],  # Toma 2: Actores 2 y 3
    [1, 0, 1, 0],  # Toma 3: Actores 1 y 3
    [0, 0, 0, 1],  # Toma 4: Actor 4
]

ejecutar_algoritmo_fuerza_bruta(matriz, 3)

ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA

Se realizarán 14 iteraciones.

Generando todas las particiones posibles...
Nueva mejor solucion encontrada - Costo: 4, Dias: 2
Tiempo: 0.00 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===

DIA 1:
Tomas: [1, 2, 3]
Actores necesarios: [1, 2, 3]
Costo del dia: 3

DIA 2:
Tomas: [4]
Actores necesarios: [4]
Costo del dia: 1

COSTO TOTAL: 4
DIAS TOTALES: 2



In [8]:
matriz = [
        [1,1,1,0,0,0],  # Toma 1
        [1,1,1,0,0,0],  # Toma 2
        [0,0,1,0,1,0],  # Toma 3
        [0,0,0,0,1,1],  # Toma 4
        [0,1,0,0,0,1],  # Toma 5
        [0,1,1,0,0,0],  # Toma 6
        [0,1,1,0,0,0],  # Toma 7
        [0,0,0,1,0,0],  # Toma 8
]

ejecutar_algoritmo_fuerza_bruta(matriz, 6)

ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA

Se realizarán 4131 iteraciones.

Generando todas las particiones posibles...
Nueva mejor solucion encontrada - Costo: 8, Dias: 2
Tiempo: 0.05 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===

DIA 1:
Tomas: [1, 2, 3, 4, 5, 6]
Actores necesarios: [1, 2, 3, 5, 6]
Costo del dia: 5

DIA 2:
Tomas: [7, 8]
Actores necesarios: [2, 3, 4]
Costo del dia: 3

COSTO TOTAL: 8
DIAS TOTALES: 2



---
# **Calcula la complejidad del algoritmo por fuerza bruta.**

- **Por día**:  
  $O(k \times m)$  
  donde $k$ = máximo de tomas por día, $m$ = número de actores.

- **Por partición**:  
  $O(d \times k \times m)$  
  donde $d$ = número de días en la partición.

- **En promedio (total de tomas = n)**:  
  $O(n \times m)$  
  ya que $\sum$ tomas por día $= n$.

$$
\boxed{T(n, k, m) = O(P(n, k) \times n \times m)}
$$

Donde:
- $P(n, k)$ = número de particiones posibles de $n$ tomas en grupos de a lo sumo $k$ elementos por grupo.


---
# **Diseña un algoritmo que mejore la complejidad del algorito por fuerza bruta.**

## Clase ProgramacionDoblaje

Esta es la clase principal que representa una solución completa del problema.


Almacena:
- La matriz de actores por tomas (qué actores participan en cada toma)
- La programación actual (qué tomas se graban cada día)
- Las métricas de calidad (número de convocatorias de actores y días totales)


Sus métodos principales incluyen:
- `obtener_programacion_inicial_heuristica()`: Genera una solución inicial inteligente agrupando tomas que comparten actores, priorizando las tomas con más actores
- `calcular_metricas()`: Calcula el costo total sumando cuántos actores únicos se necesitan cada día
- `es_mejor()`: Compara soluciones priorizando menos convocatorias de actores, y en caso de empate, menos días
- Métodos auxiliares para clonar, comparar y manipular programaciones

In [9]:
import numpy as np
import time
from copy import deepcopy
from collections import defaultdict

class ProgramacionDoblaje:
    def __init__(self, actores_tomas, num_actores=10, num_tomas=30, max_tomas_dia=6):
        self.actores_tomas = actores_tomas  # Matriz 30x10 (tomas x actores)
        self.num_actores = num_actores
        self.num_tomas = num_tomas
        self.max_tomas_dia = max_tomas_dia
        self.programacion = []  # Lista de días, cada día es una lista de tomas
        self.convocatorias = 0  # Número total de convocatorias de actores
        self.divisionesTake = 0  # Número de divisiones de tomas

    def obtener_programacion_inicial_heuristica(self):
        """Genera una programación inicial usando heurística voraz inteligente"""
        # Calcular solapamiento entre tomas (actores en común)
        solapamientos = self._calcular_matriz_solapamiento()

        # Obtener tomas ordenadas por número de actores (más actores primero)
        tomas_por_actores = [(i, sum(self.actores_tomas[i])) for i in range(self.num_tomas)]
        tomas_por_actores.sort(key=lambda x: x[1], reverse=True)

        self.programacion = []
        tomas_asignadas = set()

        for toma_principal, _ in tomas_por_actores:
            if toma_principal in tomas_asignadas:
                continue

            # Crear nuevo día comenzando con esta toma
            dia_actual = [toma_principal]
            tomas_asignadas.add(toma_principal)

            # Encontrar las mejores tomas para completar este día
            while len(dia_actual) < self.max_tomas_dia:
                mejor_toma = self._encontrar_mejor_toma_para_dia(
                    dia_actual, tomas_asignadas, solapamientos
                )
                if mejor_toma is None:
                    break

                dia_actual.append(mejor_toma)
                tomas_asignadas.add(mejor_toma)

            self.programacion.append(dia_actual)

        self.calcular_metricas()
        return self

    def _calcular_matriz_solapamiento(self):
        """Calcula cuántos actores comparten cada par de tomas"""
        solapamientos = {}
        for i in range(self.num_tomas):
            for j in range(i + 1, self.num_tomas):
                solapamiento = sum(1 for k in range(self.num_actores)
                                 if self.actores_tomas[i][k] == 1 and self.actores_tomas[j][k] == 1)
                solapamientos[(i, j)] = solapamiento
                solapamientos[(j, i)] = solapamiento
        return solapamientos

    def _encontrar_mejor_toma_para_dia(self, dia_actual, tomas_asignadas, solapamientos):
        """Encuentra la mejor toma para añadir al día actual"""
        mejor_toma = None
        mejor_score = -1

        actores_dia_actual = self._obtener_actores_dia(dia_actual)

        for toma in range(self.num_tomas):
            if toma in tomas_asignadas:
                continue

            # Calcular score basado en solapamiento con tomas del día
            score = 0
            for toma_dia in dia_actual:
                if (toma, toma_dia) in solapamientos:
                    score += solapamientos[(toma, toma_dia)]

            # Bonus por actores ya presentes en el día
            actores_toma = set(i for i in range(self.num_actores)
                             if self.actores_tomas[toma][i] == 1)
            actores_compartidos = len(actores_dia_actual.intersection(actores_toma))
            score += actores_compartidos * 2  # Peso mayor para actores ya presentes

            if score > mejor_score:
                mejor_score = score
                mejor_toma = toma

        return mejor_toma

    def _obtener_actores_dia(self, tomas_dia):
        """Obtiene el conjunto de actores necesarios para un día"""
        actores = set()
        for toma in tomas_dia:
            for actor in range(self.num_actores):
                if self.actores_tomas[toma][actor] == 1:
                    actores.add(actor)
        return actores

    def calcular_metricas(self):
        """Calcula las métricas de la programación actual"""
        self.convocatorias = 0
        self.divisionesTake = len(self.programacion)  # Número de días

        for dia in self.programacion:
            actores_dia = set()
            for toma in dia:
                for actor in range(self.num_actores):
                    if self.actores_tomas[toma][actor] == 1:
                        actores_dia.add(actor)
            self.convocatorias += len(actores_dia)

    def es_mejor(self, otra):
        """Determina si esta programación es mejor que otra"""
        if self.convocatorias < otra.convocatorias:
            return True
        elif self.convocatorias == otra.convocatorias:
            return self.divisionesTake < otra.divisionesTake
        return False

    def son_iguales(self, otra):
        """Verifica si dos programaciones son iguales"""
        if len(self.programacion) != len(otra.programacion):
            return False

        # Convertir a conjuntos ordenados para comparar
        prog1_sorted = [sorted(dia) for dia in self.programacion]
        prog2_sorted = [sorted(dia) for dia in otra.programacion]
        prog1_sorted.sort()
        prog2_sorted.sort()

        return prog1_sorted == prog2_sorted

    def clonar(self):
        """Crea una copia de la programación"""
        nueva = ProgramacionDoblaje(self.actores_tomas, self.num_actores, self.num_tomas, self.max_tomas_dia)
        nueva.programacion = deepcopy(self.programacion)
        nueva.convocatorias = self.convocatorias
        nueva.divisionesTake = self.divisionesTake
        return nueva

## Clase Movimiento

Representa una operación específica que se puede aplicar a una programación para crear una solución vecina. Encapsula:
- El tipo de movimiento (intercambiar tomas entre días o mover una toma a otro día)
- Los parámetros específicos del movimiento
- Un sistema de evaluación que estima el beneficio sin ejecutar completamente el movimiento

Incluye `calcular_beneficio_estimado()` que predice cuánto mejorará la solución, permitiendo priorizar los movimientos más prometedores antes de ejecutarlos completamente.

In [10]:
class Movimiento:
    def __init__(self, tipo, parametros, programacion_base):
        self.tipo = tipo
        self.parametros = parametros
        self.programacion_base = programacion_base
        self.resultado = None
        self.beneficio_estimado = None

    def calcular_beneficio_estimado(self):
        """Calcula el beneficio estimado del movimiento sin ejecutarlo completamente"""
        if self.beneficio_estimado is not None:
            return self.beneficio_estimado

        if self.tipo == 'intercambiar':
            dia1, toma1, dia2, toma2 = self.parametros
            # Estimar cambio en convocatorias
            beneficio = self._estimar_beneficio_intercambio(dia1, toma1, dia2, toma2)
        else:
            # Para otros tipos de movimiento, beneficio neutro como antes
            beneficio = 0

        self.beneficio_estimado = beneficio
        return beneficio

    def _estimar_beneficio_intercambio(self, dia1, toma1, dia2, toma2):
        """Estima el beneficio de intercambiar dos tomas entre días"""
        prog = self.programacion_base

        # Calcular actores actuales en cada día
        actores_dia1_actual = set()
        actores_dia2_actual = set()

        for toma in prog.programacion[dia1]:
            for actor in range(prog.num_actores):
                if prog.actores_tomas[toma][actor] == 1:
                    actores_dia1_actual.add(actor)

        for toma in prog.programacion[dia2]:
            for actor in range(prog.num_actores):
                if prog.actores_tomas[toma][actor] == 1:
                    actores_dia2_actual.add(actor)

        # Calcular actores después del intercambio
        actores_dia1_nuevo = actores_dia1_actual.copy()
        actores_dia2_nuevo = actores_dia2_actual.copy()

        # Remover actores de tomas que se van
        for actor in range(prog.num_actores):
            if prog.actores_tomas[toma1][actor] == 1:
                # Verificar si el actor sigue siendo necesario en día1
                necesario = any(prog.actores_tomas[t][actor] == 1
                              for t in prog.programacion[dia1] if t != toma1)
                if not necesario:
                    actores_dia1_nuevo.discard(actor)

            if prog.actores_tomas[toma2][actor] == 1:
                necesario = any(prog.actores_tomas[t][actor] == 1
                              for t in prog.programacion[dia2] if t != toma2)
                if not necesario:
                    actores_dia2_nuevo.discard(actor)

        # Añadir actores de tomas que llegan
        for actor in range(prog.num_actores):
            if prog.actores_tomas[toma2][actor] == 1:
                actores_dia1_nuevo.add(actor)
            if prog.actores_tomas[toma1][actor] == 1:
                actores_dia2_nuevo.add(actor)

        convocatorias_antes = len(actores_dia1_actual) + len(actores_dia2_actual)
        convocatorias_despues = len(actores_dia1_nuevo) + len(actores_dia2_nuevo)

        return convocatorias_antes - convocatorias_despues

    def ejecutar(self):
        """Ejecuta el movimiento y retorna la nueva programación"""
        if self.resultado is not None:
            return self.resultado

        nueva = self.programacion_base.clonar()

        if self.tipo == 'intercambiar':
            dia1, toma1, dia2, toma2 = self.parametros
            try:
                nueva.programacion[dia1].remove(toma1)
                nueva.programacion[dia1].append(toma2)
                nueva.programacion[dia2].remove(toma2)
                nueva.programacion[dia2].append(toma1)
            except (ValueError, IndexError):
                return None

        elif self.tipo == 'mover':
            dia_origen, toma, dia_destino = self.parametros
            try:
                nueva.programacion[dia_origen].remove(toma)
                nueva.programacion[dia_destino].append(toma)
                # Eliminar días vacíos
                nueva.programacion = [dia for dia in nueva.programacion if dia]
            except (ValueError, IndexError):
                return None

        nueva.calcular_metricas()
        self.resultado = nueva
        return nueva

    def __eq__(self, other):
        if not isinstance(other, Movimiento):
            return False
        return self.tipo == other.tipo and self.parametros == other.parametros

    def __hash__(self):
        return hash((self.tipo, tuple(self.parametros) if isinstance(self.parametros, list) else self.parametros))

## Clase ListaTabu

Implementa la memoria del algoritmo tabú, manteniendo un historial de movimientos recientes para evitar ciclos. Características:
- Lista de tamaño limitado que almacena movimientos prohibidos temporalmente
- Sistema FIFO (primero en entrar, primero en salir) para gestión automática del tamaño
- Método `no_contiene()` para verificar si un movimiento está prohibido

In [11]:
class ListaTabu:
    def __init__(self, tamaño_maximo=50):
        self.lista = []
        self.tamaño_maximo = tamaño_maximo

    def crear_lista_vacia(self):
        self.lista = []
        return self

    def no_contiene(self, movimiento):
        return movimiento not in self.lista

    def añadir_movimiento(self, movimiento):
        if movimiento in self.lista:
            self.lista.remove(movimiento)

        self.lista.append(movimiento)

        if len(self.lista) > self.tamaño_maximo:
            self.lista.pop(0)

## Clase OptimizadorBusquedaTabu

Es el motor principal del algoritmo que coordina todo el proceso de optimización.


Sus responsabilidades incluyen:

1.  Generación inteligente de vecinos:
    - `_generar_intercambios_prometedores()`: Crea intercambios entre tomas que probablemente reduzcan convocatorias
    - `_generar_movimientos_consolidacion()`: Busca oportunidades para agrupar tomas con actores comunes
    - `_generar_movimientos_exploratorios()`: Añade diversidad para evitar óptimos locales

2.  Control del algoritmo:
    - Gestiona la lista tabú y criterios de aspiración
    - Implementa mecanismos de escape cuando se detecta estancamiento
    - Coordina las iteraciones y la selección del mejor movimiento en cada paso

3.  Optimizaciones de rendimiento:
    - Cache de solapamientos entre tomas para cálculos eficientes
    - Evaluación lazy de movimientos (solo calcula el beneficio cuando es necesario)

La clase utiliza heurísticas inteligentes para enfocar la búsqueda en las regiones más prometedoras del espacio de soluciones, haciendo el algoritmo mucho más eficiente que una búsqueda tabú básica.

In [12]:
class OptimizadorBusquedaTabu:
    def __init__(self, actores_tomas):
        # Parámetros del algoritmo
        self.ITERACIONES = 1000
        self.NUMERO_VECINOS = 20
        self.MAXIMO_REPETICIONES = 10

        # Datos del problema
        self.actores_tomas = actores_tomas

        # Cache para análisis de movimientos
        self.cache_solapamientos = None

    def _inicializar_cache(self, programacion):
        """Inicializa estructuras de datos para optimizar la búsqueda"""
        if self.cache_solapamientos is None:
            self.cache_solapamientos = {}
            for i in range(len(self.actores_tomas)):
                for j in range(i + 1, len(self.actores_tomas)):
                    solapamiento = sum(1 for k in range(len(self.actores_tomas[0]))
                                     if self.actores_tomas[i][k] == 1 and self.actores_tomas[j][k] == 1)
                    self.cache_solapamientos[(i, j)] = solapamiento
                    self.cache_solapamientos[(j, i)] = solapamiento

    def generar_vecinos_inteligentes(self, programacion, numero_vecinos):
        """Genera vecinos usando heurísticas inteligentes"""
        self._inicializar_cache(programacion)

        vecinos = []

        # 1. Movimientos de intercambio basados en beneficio estimado
        movimientos_intercambio = self._generar_intercambios_prometedores(programacion)
        vecinos.extend(movimientos_intercambio[:numero_vecinos//2])

        # 2. Movimientos de consolidación (juntar tomas con actores comunes)
        movimientos_consolidacion = self._generar_movimientos_consolidacion(programacion)
        vecinos.extend(movimientos_consolidacion[:numero_vecinos//3])

        # 3. Algunos movimientos exploratorios (menor proporción)
        movimientos_exploratorios = self._generar_movimientos_exploratorios(programacion)
        vecinos.extend(movimientos_exploratorios[:numero_vecinos//6])

        # Ordenar por beneficio estimado
        for mov in vecinos:
            mov.calcular_beneficio_estimado()

        vecinos.sort(key=lambda x: x.beneficio_estimado, reverse=True)

        return vecinos[:numero_vecinos]

    def _generar_intercambios_prometedores(self, programacion):
        """Genera intercambios que probablemente reduzcan convocatorias"""
        movimientos = []

        for dia1 in range(len(programacion.programacion)):
            for dia2 in range(dia1 + 1, len(programacion.programacion)):
                for toma1 in programacion.programacion[dia1]:
                    for toma2 in programacion.programacion[dia2]:
                        # Solo considerar intercambios que podrían ser beneficiosos
                        if self._intercambio_es_prometedor(toma1, toma2, dia1, dia2, programacion):
                            mov = Movimiento('intercambiar', [dia1, toma1, dia2, toma2], programacion)
                            movimientos.append(mov)

        return movimientos

    def _intercambio_es_prometedor(self, toma1, toma2, dia1, dia2, programacion):
        """Determina si un intercambio podría ser beneficioso"""
        # Obtener actores de cada toma
        actores_toma1 = set(i for i in range(len(self.actores_tomas[0]))
                           if self.actores_tomas[toma1][i] == 1)
        actores_toma2 = set(i for i in range(len(self.actores_tomas[0]))
                           if self.actores_tomas[toma2][i] == 1)

        # Obtener actores de cada día (sin las tomas a intercambiar)
        actores_dia1 = set()
        for t in programacion.programacion[dia1]:
            if t != toma1:
                for actor in range(len(self.actores_tomas[0])):
                    if self.actores_tomas[t][actor] == 1:
                        actores_dia1.add(actor)

        actores_dia2 = set()
        for t in programacion.programacion[dia2]:
            if t != toma2:
                for actor in range(len(self.actores_tomas[0])):
                    if self.actores_tomas[t][actor] == 1:
                        actores_dia2.add(actor)

        # El intercambio es prometedor si aumenta el solapamiento
        solapamiento_actual = (len(actores_toma1.intersection(actores_dia1)) +
                              len(actores_toma2.intersection(actores_dia2)))
        solapamiento_nuevo = (len(actores_toma2.intersection(actores_dia1)) +
                             len(actores_toma1.intersection(actores_dia2)))

        return solapamiento_nuevo > solapamiento_actual

    def _generar_movimientos_consolidacion(self, programacion):
        """Genera movimientos para consolidar actores comunes"""
        movimientos = []

        # Buscar tomas con actores en común que estén en días diferentes
        for dia1 in range(len(programacion.programacion)):
            for dia2 in range(len(programacion.programacion)):
                if dia1 == dia2 or len(programacion.programacion[dia2]) >= programacion.max_tomas_dia:
                    continue

                for toma in programacion.programacion[dia1]:
                    if self._movimiento_consolida_actores(toma, dia1, dia2, programacion):
                        mov = Movimiento('mover', [dia1, toma, dia2], programacion)
                        movimientos.append(mov)

        return movimientos

    def _movimiento_consolida_actores(self, toma, dia_origen, dia_destino, programacion):
        """Verifica si mover una toma consolida actores"""
        actores_toma = set(i for i in range(len(self.actores_tomas[0]))
                          if self.actores_tomas[toma][i] == 1)

        actores_dia_destino = set()
        for t in programacion.programacion[dia_destino]:
            for actor in range(len(self.actores_tomas[0])):
                if self.actores_tomas[t][actor] == 1:
                    actores_dia_destino.add(actor)

        # El movimiento es bueno si hay solapamiento de actores
        return len(actores_toma.intersection(actores_dia_destino)) > 0

    def _generar_movimientos_exploratorios(self, programacion):
        """Genera algunos movimientos exploratorios para evitar óptimos locales"""
        movimientos = []

        # Seleccionar algunos intercambios al azar pero priorizando días con más actores
        dias_por_actores = []
        for i, dia in enumerate(programacion.programacion):
            actores_dia = set()
            for toma in dia:
                for actor in range(len(self.actores_tomas[0])):
                    if self.actores_tomas[toma][actor] == 1:
                        actores_dia.add(actor)
            dias_por_actores.append((i, len(actores_dia)))

        # Ordenar por número de actores (más actores primero)
        dias_por_actores.sort(key=lambda x: x[1], reverse=True)

        # Generar algunos intercambios entre días con muchos actores
        for i in range(min(3, len(dias_por_actores))):
            for j in range(i + 1, min(6, len(dias_por_actores))):
                dia1, _ = dias_por_actores[i]
                dia2, _ = dias_por_actores[j]

                if programacion.programacion[dia1] and programacion.programacion[dia2]:
                    toma1 = programacion.programacion[dia1][0]  # Primera toma del día
                    toma2 = programacion.programacion[dia2][0]  # Primera toma del día
                    mov = Movimiento('intercambiar', [dia1, toma1, dia2, toma2], programacion)
                    movimientos.append(mov)

        return movimientos

    def elegir_mejor(self, vecinos):
        """Elige el mejor movimiento de la lista de vecinos"""
        if not vecinos:
            return None

        mejor_movimiento = None
        mejor_programacion = None

        for movimiento in vecinos:
            programacion_resultado = movimiento.ejecutar()
            if programacion_resultado is not None:
                if mejor_programacion is None or programacion_resultado.es_mejor(mejor_programacion):
                    mejor_movimiento = movimiento
                    mejor_programacion = programacion_resultado

        return mejor_movimiento

    def cumple_aspiracion(self, movimiento, mejor_global):
        """Criterio de aspiración: acepta movimientos tabú si mejoran el mejor global"""
        programacion_resultado = movimiento.ejecutar()
        if programacion_resultado is None:
            return False
        return programacion_resultado.es_mejor(mejor_global)

    def ejecutar_mecanismo_escape_simple(self, programacion):
        """Mecanismo de escape simplificado"""
        nueva = programacion.clonar()

        # Intercambiar las primeras tomas de los dos primeros días
        if len(nueva.programacion) >= 2 and nueva.programacion[0] and nueva.programacion[1]:
            toma1 = nueva.programacion[0][0]
            toma2 = nueva.programacion[1][0]

            nueva.programacion[0].remove(toma1)
            nueva.programacion[0].append(toma2)
            nueva.programacion[1].remove(toma2)
            nueva.programacion[1].append(toma1)

        nueva.calcular_metricas()
        return nueva

    def optimizar(self, max_tomas_dia=6):
        """Ejecuta el algoritmo de búsqueda tabú con heurísticas inteligentes"""
        # Inicialización con heurística
        num_tomas = len(self.actores_tomas)
        num_actores = len(self.actores_tomas[0]) if self.actores_tomas else 0

        actual = ProgramacionDoblaje(self.actores_tomas, num_actores, num_tomas, max_tomas_dia)
        actual.obtener_programacion_inicial_heuristica()  # Usar inicialización heurística

        mejor = actual.clonar()
        repeticiones = 0
        lista_tabu = ListaTabu().crear_lista_vacia()

        print(f"Solución inicial: {actual.convocatorias} convocatorias, {actual.divisionesTake} días")

        for i in range(self.ITERACIONES):
            if i % 100 == 0:
                print(f"Iteraciones: {i:}, Mejor costo: {mejor.convocatorias}")

            vecinos = self.generar_vecinos_inteligentes(actual, self.NUMERO_VECINOS)
            mejor_movimiento = None

            while mejor_movimiento is None and vecinos:
                mejor_movimiento = self.elegir_mejor(vecinos)

                if mejor_movimiento is not None:
                    if (lista_tabu.no_contiene(mejor_movimiento) or
                        self.cumple_aspiracion(mejor_movimiento, mejor)):
                        lista_tabu.añadir_movimiento(mejor_movimiento)
                    else:
                        vecinos.remove(mejor_movimiento)
                        mejor_movimiento = None

            if mejor_movimiento is not None:
                nueva = mejor_movimiento.ejecutar()

                if nueva is not None:
                    if nueva.son_iguales(actual):
                        repeticiones += 1
                        if repeticiones == self.MAXIMO_REPETICIONES:
                            actual = self.ejecutar_mecanismo_escape_simple(actual)
                            repeticiones = 0
                        else:
                            actual = nueva
                    else:
                        actual = nueva
                        repeticiones = 0

                    if actual.es_mejor(mejor):
                        mejor = actual.clonar()
                        print(f"Nueva mejor solucion encontrada - Costo: {mejor.convocatorias}, Dias: {mejor.divisionesTake}")

        return mejor

## Función `mostrar_resultados()`

Es la función de presentación que formatea y muestra la solución final de manera clara y legible. Básicamente convierte la representación interna de la solución en un formato amigable para que el usuario pueda interpretar fácilmente la planificación óptima.

## Función `ejecutar_algoritmo_tabu_inteligente()`

Es la función principal que orquesta todo el proceso de optimización de principio a fin. Es esencialmente el punto de entrada principal que conecta todos los componentes del sistema y proporciona una interfaz limpia para ejecutar la optimización completa con una sola llamada de función.

In [13]:
def mostrar_resultados(solucion):
    """Muestra los resultados de la optimización"""
    print("\n=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR ALGORITMO TABÚ ===")

    for i, dia in enumerate(solucion.programacion, 1):
        print(f"\nDIA {i}:")
        print(f"Tomas: {[t+1 for t in sorted(dia)]}")

        # Calcular actores necesarios para este día
        actores_dia = set()
        for toma in dia:
            for actor in range(solucion.num_actores):
                if solucion.actores_tomas[toma][actor] == 1:
                    actores_dia.add(actor + 1)  # +1 para numeración desde 1
        print(f"Actores necesarios: {sorted(list(actores_dia))}")
        print(f"Costo del dia: {len(actores_dia)}")

    print(f"\nCOSTO TOTAL: {solucion.convocatorias}")
    print(f"DIAS TOTALES: {solucion.divisionesTake}\n")

def ejecutar_algoritmo_tabu_inteligente(matriz_enunciado, max_tomas_por_dia_enunciado):
    """
    Función principal para ejecutar el algoritmo de búsqueda tabú con heurísticas inteligentes.
    """
    print("="*70)
    print("ORGANIZAR SESIONES DE DOBLAJE - BÚSQUEDA TABÚ INTELIGENTE")
    print("="*70)

    # Crear optimizador con los parámetros proporcionados
    tiempo_inicio = time.time()
    optimizador = OptimizadorBusquedaTabu(matriz_enunciado)

    print("Ejecutando algoritmo de búsqueda tabú con heurísticas inteligentes...")

    # Ejecutar optimización
    mejor_solucion = optimizador.optimizar(max_tomas_por_dia_enunciado)
    tiempo_total = time.time() - tiempo_inicio
    print(f"Tiempo: {tiempo_total:.2f} segundos")

    # Mostrar resultados
    mostrar_resultados(mejor_solucion)
    return mejor_solucion

---
# **Aplicar al algoritmo optimizado la matriz y restricciones del enunciado**

In [14]:
costo = ejecutar_algoritmo_tabu_inteligente(matriz_enunciado, max_tomas_por_dia_enunciado)

ORGANIZAR SESIONES DE DOBLAJE - BÚSQUEDA TABÚ INTELIGENTE
Ejecutando algoritmo de búsqueda tabú con heurísticas inteligentes...
Solución inicial: 32 convocatorias, 5 días
Iteraciones: 0, Mejor costo: 32
Nueva mejor solucion encontrada - Costo: 30, Dias: 5
Nueva mejor solucion encontrada - Costo: 29, Dias: 5
Nueva mejor solucion encontrada - Costo: 28, Dias: 5
Iteraciones: 100, Mejor costo: 28
Iteraciones: 200, Mejor costo: 28
Nueva mejor solucion encontrada - Costo: 27, Dias: 5
Iteraciones: 300, Mejor costo: 27
Iteraciones: 400, Mejor costo: 27
Iteraciones: 500, Mejor costo: 27
Iteraciones: 600, Mejor costo: 27
Iteraciones: 700, Mejor costo: 27
Iteraciones: 800, Mejor costo: 27
Iteraciones: 900, Mejor costo: 27
Tiempo: 9.91 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR ALGORITMO TABÚ ===

DIA 1:
Tomas: [7, 9, 13, 27, 28, 30]
Actores necesarios: [1, 2, 4, 5]
Costo del dia: 4

DIA 2:
Tomas: [5, 6, 10, 11, 12, 26]
Actores necesarios: [1, 2, 3, 4, 5, 6, 8, 9]
Costo del dia:

---
# **¿Cuál es la estructura de datos que mejor se adapta al problema? Argumenta la respuesta.**

### 1. Matriz bidimensional (Lista de Listas) para `actores_tomas`
```python
self.actores_tomas = actores_tomas  # Matriz 30x10 (tomas x actores)
```

¿Por qué es la mejor opción?
- Acceso O(1): Permite verificar instantáneamente si un actor participa en una toma específica
- Representación natural: La relación toma-actor es inherentemente matricial
- Eficiencia en cálculos: Facilita operaciones como calcular solapamientos entre tomas
- Memoria predecible: Estructura fija que optimiza el uso de memoria

### 2. Lista de listas para `programacion`
```python
self.programacion = []  # Lista de días, cada día es una lista de tomas
```

Ventajas de esta estructura:
- Flexibilidad: Permite días con diferente número de tomas
- Iteración eficiente: Fácil recorrido por días y tomas
- Operaciones dinámicas: Inserción/eliminación de tomas es O(1) al final, O(n) en posiciones específicas
- Representación intuitiva: Modela naturalmente la realidad (días → tomas)

### 3. Diccionario para cache de solapamientos
```python
self.cache_solapamientos = {}
# Almacena: {(toma_i, toma_j): num_actores_comunes}
```

¿Por qué es óptima?
- Acceso O(1): Búsqueda instantánea de solapamientos precalculados
- Evita recálculos: Los solapamientos se calculan una vez y se reutilizan
- Optimización crucial: Sin esta cache, cada cálculo de solapamiento sería O(num_actores)

### 4. Conjuntos (Sets) para manejo de actores
```python
actores_dia = set()
actores_toma = set(i for i in range(num_actores) if self.actores_tomas[toma][i] == 1)
```

Beneficios clave:
- Operaciones set O(1) promedio: Intersección, unión, diferencia son muy eficientes
- Eliminación de duplicados: Automática, crucial para contar actores únicos
- Operaciones lógicas: Perfectas para calcular actores en común entre días/tomas

### 5. Lista circular para lista Tabú
```python
class ListaTabu:
    def __init__(self, tamaño_maximo=50):
        self.lista = []
```

Por qué funciona bien:
- FIFO eficiente: Mantiene los movimientos más recientes
- Búsqueda O(n): Acceptable dado el tamaño limitado (50 elementos)
- Inserción O(1): Al final de la lista

### La combinación de estructuras elegida es óptima para este problema porque:

1. Matriz para relaciones fijas (actores-tomas): Máxima eficiencia en consultas constantes
2. Listas para secuencias dinámicas (programación): Balance perfecto entre flexibilidad y eficiencia
3. Diccionarios para cachés: Evitan recálculos costosos
4. Sets para operaciones de conjuntos: Naturales para manejar grupos de actores
5. Lista simple para tabú: Adecuada para el tamaño limitado y acceso secuencial

Esta arquitectura logra un balance óptimo entre eficiencia temporal, uso de memoria y simplicidad de implementación, siendo especialmente efectiva para las operaciones críticas del algoritmo de búsqueda tabú.

---
# **Calcula la complejidad del algoritmo**

A continuación se muestra un análisis aproximado de la complejidad temporal del algoritmo, en función de:

* $n$ = número de tomas
* $m$ = número de actores
* $k$ = máximo de tomas por día
* $I$ = número de iteraciones de la búsqueda tabú (parámetro `ITERACIONES`)
* $B$ = número de vecinos generados en cada iteración (parámetro `NUMERO_VECINOS`)

### 1. Inicialización heurística

#### 1.1. Cálculo de la matriz de solapamiento

   * Recorre todos los pares de tomas: $\mathcal{O}(n^2)$
   * Para cada par, comprueba solapamiento en los $m$ actores: $\mathcal{O}(m)$
   * Total:

     $$
       \mathcal{O}(n^2 \,m)
     $$

#### 1.2. Ordenación de tomas por número de actores

   * Suma de actores por toma: $\mathcal{O}(n\,m)$
   * Ordenación de $n$ elementos: $\mathcal{O}(n\log n)$
   * Total: $\mathcal{O}(n\,m + n\log n)$, pero queda dominado por los términos cuadrados.

#### 1.3. Construcción voraz de días

   * Para cada toma (hasta $n$), se añaden hasta $k$ tomas más:

     * Cada búsqueda escanea hasta $n$ candidatas, con cálculo de score en $\mathcal{O}(k + m)$.
   * Total aproximado:

       \begin{align*}
       \sum_{i=1}^{n} \mathcal{O}(n(k + m))
       &= \mathcal{O}(n^2(k + m)) \\
       &\approx \mathcal{O}(n^2 k + n^2 m)
       \end{align*}

Coste total de inicialización:

$$
\boxed{
  \mathcal{O}\bigl(n^2m + n^2k\bigr)
}
$$

### 2. Cada iteración de la búsqueda tabú

Por cada una de las $I$ iteraciones:

#### 2.1 Generación de vecinos inteligentes

* Intercambios prometedores

  * Se exploran pares de días (≈ $(n/k)^2$) y pares de tomas por día (≈ $k^2$), evaluando solapamiento en $\mathcal{O}(m)$.
  * Coste:

    $$
      \[
      \mathcal{O}\bigl((n/k)^2 \cdot k^2 \cdot m\bigr)
      = \mathcal{O}(n^2 m)
      \]
    $$

* Movimientos de consolidación

  * Para cada par de días y cada toma ($k$), comprueba actores en $\mathcal{O}(m)$.
  * Coste:

    $$
      \[
      \mathcal{O}\left(\left(\frac{n}{k}\right)^2 \cdot k \cdot m\right)
      = \mathcal{O}\left(\frac{n^2 m}{k}\right)
      \]
    $$

* Movimientos exploratorios

  * Cálculo de actores por día ($\mathcal{O}((n/k)\,k\,m)=\mathcal{O}(n\,m)$), ordenación de días ($\mathcal{O}((n/k)\log(n/k))$) y un número pequeño de intercambios.
  * Coste menor, $\mathcal{O}(n\,m + (n/k)\log(n/k))$.

* Total generación de vecinos (dominante):

  $$
    \boxed{
      \mathcal{O}(n^2\,m)
    }
  $$

  (pues $n^2 m \gg n^2 m/k$ y $\gg n m$ cuando $k$ es constante.)

#### 2.2 Evaluación y selección de vecinos

* Para cada uno de los $B$ vecinos:

  1. Clonación de la programación: $\mathcal{O}(n)$
  2. Recálculo de métricas (actores por día): $\mathcal{O}((n/k)\,k\,m)=\mathcal{O}(n\,m)$
* Total evaluación:

  $$
    \boxed{
      \mathcal{O}\bigl(B(n + n m)\bigr)
      \approx
      \mathcal{O}(B n m)
    }
  $$

#### 2.3 Gestión de la lista tabú

* Operaciones en lista de tamaño fijo: $O(1)$ amortizado.

### 3. Complejidad por iteración

Sumando generación y evaluación:

$$
\boxed{
  \mathcal{O}\bigl(n^2\,m + B\,n\,m\bigr)
}
$$

Como normalmente $B \ll n$, el término dominante es $O(n^2 m)$.

### 4. Complejidad total del algoritmo

Añadiendo las $I$ iteraciones más la inicialización:

$$
\boxed{
  \underbrace{O(n^2 m + n^2 k)}_{\text{inicialización}} +
  I \times \underbrace{O(n^2 m + B\,n\,m)}_{\text{bucle}}
}
\approx
O(n^2 m + n^2 k + I\,n^2 m + I\,B\,n\,m)
$$

### 5. Caso típico y término dominante

En la práctica:

* Se suele cumplir $k \ll n$.
* Y $B$ es pequeño comparado con $n$.

Por tanto:

1. Inicialización queda dominada por $O(n^2 m)$.
2. Cada iteración queda dominada por $O(n^2 m)$.
3. Total:

   $$
     O\bigl(n^2 m + I\,n^2 m\bigr)
     \;=\;
     O\bigl(I\,n^2\,m\bigr)
   $$

El algoritmo crece cuadráticamente en el número de tomas $n$, linealmente en el número de actores $m$ y en las iteraciones $I$.

---
# **Argumenta porque crees que mejora el algoritmo por fuerza bruta.**

## 1. Complejidad computacional Exponencial vs Polinomial

### Fuerza Bruta:
- **Complejidad**: T(n,k,m) = O(P(n,k) × n × m) donde:
  - n = número de tomas (30)
  - k = máximo tomas por día (6)
  - m = número de actores (10)
  - P(n,k) = particiones de n tomas en grupos de máximo k elementos
- **Tiempo estimado**: Computacionalmente inviable

### Búsqueda Tabú:
- **Complejidad total**: O(I × n² × m) donde:
  - I = iteraciones (1000)
  - n = número de tomas (30)
  - m = número de actores (10)
- **Tiempo real**: Segundos o minutos

## 2. Inicialización inteligente vs Enumeración ciega

### Algoritmo Tabú:
```python
def obtener_programacion_inicial_heuristica(self):
    # Calcula solapamientos entre tomas (actores en común)
    solapamientos = self._calcular_matriz_solapamiento()
    
    # Ordena tomas por número de actores (más actores primero)
    tomas_por_actores = [(i, sum(self.actores_tomas[i])) for i in range(self.num_tomas)]
    tomas_por_actores.sort(key=lambda x: x[1], reverse=True)
```

**Ventaja**: Comienza con una solución ya optimizada usando conocimiento del dominio.

### Fuerza Bruta:
- Debe evaluar **todas** las combinaciones posibles sin aprovechar patrones
- No utiliza información sobre la estructura del problema

## 3. Búsqueda dirigida vs Exploración exhaustiva

### Heurísticas inteligentes del algoritmo Tabú:

#### A) **Movimientos Prometedores**:
```python
def _intercambio_es_prometedor(self, toma1, toma2, dia1, dia2, programacion):
    # Solo considera intercambios que aumentan el solapamiento de actores
    solapamiento_nuevo > solapamiento_actual
```

#### B) **Consolidación de actores**:
```python
def _generar_movimientos_consolidacion(self, programacion):
    # Busca tomas con actores en común para agruparlas
    return len(actores_toma.intersection(actores_dia_destino)) > 0
```

#### C) **Evaluación de beneficio estimado**:
```python
def calcular_beneficio_estimado(self):
    # Estima el beneficio SIN ejecutar completamente el movimiento
    beneficio = convocatorias_antes - convocatorias_despues
```

**Resultado**: Cada movimiento está **dirigido hacia la mejora** del objetivo.

## 4. Mecanismo de escape anti-óptimos locales

### Problema de óptimos locales:
- **Fuerza Bruta**: No sufre de óptimos locales, pero es computacionalmente inviable
- **Búsqueda Local Simple**: Se queda atrapada en soluciones subóptimas

### Solución del algoritmo Tabú:
```python
def ejecutar_mecanismo_escape_inteligente(self, programacion):
    # Identifica el día con más actores
    # Redistribuye tomas estratégicamente
    # Permite explorar nuevas regiones del espacio de soluciones
```

**Ventaja**: Combina la eficiencia computacional con la capacidad de escape.

## 5. Uso eficiente de memoria y cache

### Optimizaciones de rendimiento:
```python
def _inicializar_cache(self, programacion):
    # Pre-calcula solapamientos entre tomas
    self.cache_solapamientos[(i, j)] = solapamiento
    
def calcular_beneficio_estimado(self):
    # Almacena beneficios calculados para evitar recálculos
    if self.beneficio_estimado is not None:
        return self.beneficio_estimado
```

**Resultado**: Evita cálculos redundantes que en fuerza bruta se repetirían millones de veces.

## 6. Criterios de aspiración inteligentes

```python
def cumple_aspiracion(self, movimiento, mejor_global):
    # Acepta movimientos tabú si mejoran la mejor solución global
    return programacion_resultado.es_mejor(mejor_global)
```

**Ventaja**: Permite flexibilidad estratégica manteniendo la dirección hacia el óptimo.

---
# Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorio.

In [15]:
import random

def generar_datos_aleatorios():
    # Configuración aleatoria
    tomas = random.randint(6, 12)
    actores = random.randint(5, 10)
    max_tomas_por_dia = random.randint(3, 10)

    print(f"Generando: {tomas} tomas, {actores} actores")

    # Generar matriz de actores
    matriz = []
    for i in range(tomas):
        fila = [random.randint(0, 1) for _ in range(actores)]
        # Asegurar que al menos un actor esté en cada toma
        if sum(fila) == 0:
            fila[random.randint(0, actores-1)] = 1
        matriz.append(fila)

    # Mostrar resultados
    print("matriz = [")
    for i, fila in enumerate(matriz):
        print(f"        {fila},  # Toma {i+1}")
    print("]")

    print(f"\nmax_tomas_por_dia = {max_tomas_por_dia}")

    total_iteraciones = total_posibilidades_sin_orden(len(matriz), max_tomas_por_dia)
    print(f"\nSe necesitan {total_iteraciones} iteraciones de fuerza bruta.")

    return matriz, max_tomas_por_dia

matriz, max_tomas_por_dia = generar_datos_aleatorios()

def generar_datos_y_ejecutar_ambos_algoritmos():
    ejecutar_algoritmo_fuerza_bruta(matriz, max_tomas_por_dia)
    ejecutar_algoritmo_tabu_inteligente(matriz, max_tomas_por_dia)

Generando: 10 tomas, 6 actores
matriz = [
        [0, 0, 0, 0, 1, 0],  # Toma 1
        [0, 0, 0, 1, 1, 0],  # Toma 2
        [0, 0, 0, 0, 0, 1],  # Toma 3
        [0, 1, 1, 0, 1, 0],  # Toma 4
        [1, 1, 0, 0, 1, 1],  # Toma 5
        [0, 1, 0, 1, 0, 1],  # Toma 6
        [1, 0, 1, 0, 1, 0],  # Toma 7
        [0, 1, 0, 1, 0, 1],  # Toma 8
        [1, 0, 1, 0, 0, 0],  # Toma 9
        [0, 0, 0, 0, 0, 1],  # Toma 10
]

max_tomas_por_dia = 5

Se necesitan 112124 iteraciones de fuerza bruta.


---
# Aplica el algoritmo al juego de datos aleatorio generado.

In [16]:
generar_datos_y_ejecutar_ambos_algoritmos()

ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA

Se realizarán 112124 iteraciones.

Generando todas las particiones posibles...
Nueva mejor solucion encontrada - Costo: 12, Dias: 2
Nueva mejor solucion encontrada - Costo: 11, Dias: 2
Nueva mejor solucion encontrada - Costo: 9, Dias: 2
Iteraciones: 100000, Mejor costo: 9
Tiempo: 0.84 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===

DIA 1:
Tomas: [1, 2, 3, 6, 8]
Actores necesarios: [2, 4, 5, 6]
Costo del dia: 4

DIA 2:
Tomas: [4, 5, 7, 9, 10]
Actores necesarios: [1, 2, 3, 5, 6]
Costo del dia: 5

COSTO TOTAL: 9
DIAS TOTALES: 2

ORGANIZAR SESIONES DE DOBLAJE - BÚSQUEDA TABÚ INTELIGENTE
Ejecutando algoritmo de búsqueda tabú con heurísticas inteligentes...
Solución inicial: 10 convocatorias, 2 días
Iteraciones: 0, Mejor costo: 10
Nueva mejor solucion encontrada - Costo: 9, Dias: 2
Iteraciones: 100, Mejor costo: 9
Iteraciones: 200, Mejor costo: 9
Iteraciones: 300, Mejor costo: 9
Iteraciones: 400, Mejor costo: 9
Ite

---
# Describe brevemente en unas líneas como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño.


## 1. Algoritmos híbridos y paralelos
- Búsqueda Tabú + Algoritmos Genéticos: Usar poblaciones de soluciones con operadores genéticos optimizados
- Paralelización: Evaluar múltiples vecindarios simultáneamente, especialmente crítico para instancias grandes (100+ tomas)
- Machine Learning: Entrenar modelos para predecir movimientos prometedores y reducir espacio de búsqueda

## 2. Variaciones realistas del problema
- Costos diferenciados: Actores con tarifas variables, penalizaciones por días no consecutivos
- Restricciones temporales: Disponibilidad limitada de actores, fechas de entrega
- Calidad de grabación: Minimizar cambios de configuración técnica entre tomas
- Recursos limitados: Estudios múltiples, equipos técnicos especializados

## 3. Problemas relacionados
- Scheduling cinematográfico: Locaciones, clima, disponibilidad de equipos
- Planificación de conciertos: Músicos, instrumentos, repertorio
- Organización de conferencias: Ponentes, salas, audiencias específicas

## 4. Métricas avanzadas
- Análisis de robustez: Sensibilidad ante cambios de última hora
- Equidad: Distribución balanceada de carga de trabajo entre actores
- Sostenibilidad: Minimizar desplazamientos y impacto ambiental

---
# Bibliografía utilizada

- CALDAS LIMA, Alberto. Aplicación de algoritmos heurísticos para optimizar el coste de doblaje de películas. A Coruña, 2014. Proyecto fin de máster. Universidad de A Coruña. Directores: Silvia María Lorenzo Freire, María Luisa Carpente Rodríguez. [En línea]. Disponible en: http://eio.usc.es/pub/mte/descargas/ProyectosFinMaster/Proyecto_759.pdf